In [5]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

True

In [6]:
MODEL_NAME = 'gpt-4'
filename = '../../data/ndis/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf'
DIRECTORY = '../../data/ndis'
ACTIVELOOP_DATASET_NAME = 'ndis_price_guide'

In [1]:
import tiktoken

def concatenate_page_contents(input_documents, separator):
    page_contents = [doc.dict()['page_content'] for doc in input_documents]
    return separator.join(page_contents)

def num_tokens_from_string(string: str, model_name: str = MODEL_NAME) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

NameError: name 'MODEL_NAME' is not defined

In [4]:
import os
from langchain.document_loaders import CSVLoader
from langchain.document_loaders.pdf import PDFPlumberLoader

class MyDirectoryLoader:

    def __init__(self, dir_path):
        self.dir_path = dir_path
        
    def check_args(self):
        print(**self.pdf_args.keys())

    def load(self):
        docs = []
        for root, _, files in os.walk(self.dir_path):
            for file in files:
                print('file:', file)
                file_path = os.path.join(root, file)
                if file_path.endswith('.csv'):
                    loader = CSVLoader(file_path)
                elif file_path.endswith('.pdf'):
                    loader = PDFPlumberLoader(file_path)
                else:
                    print(f"Do not process the file: {file_path}")
                    continue
                loaded_docs = loader.load()
                docs.extend(loaded_docs)
        return docs

In [5]:
loader = MyDirectoryLoader(DIRECTORY)
docs = loader.load()
docs

file: NDIS Pricing Arrangements for Specialist Disability Accommodation 2023-24  - effective 1 July 2023.pdf
file: PB Group-Based Supports Transition Guidance.pdf
file: PB NDIS Assistive Technology, Home Modifications and Consumables Code Guide 2023-24 v1.0.docx.pdf
file: PB NDIS Bereavement Addendum 2023-24 v1.0.pdf
file: PB NDIS COVID Addendum 2023-24 v1.0.pdf
file: PB NDIS Disability Support Worker Cost Model Assumptions and Methodology 2023-24 v1.0.docx.pdf
file: PB NDIS Home and Living Demonstration Projects Addendum 2023-24 v1.0.docx.pdf
file: PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf
file: PB NDIS Support Catalogue 2023-24 .xlsx
Do not process the file: ../../data/ndis/PB NDIS Support Catalogue 2023-24 .xlsx


[Document(page_content='National Disability Insurance Scheme\nPricing Arrangements\nfor\nSpecialist Disability Accommodation\n2023-24\nValid from: 1 July 2023\nVersion 1.2\n(Released 12 July 2023)', metadata={'source': '../../data/ndis/NDIS Pricing Arrangements for Specialist Disability Accommodation 2023-24  - effective 1 July 2023.pdf', 'file_path': '../../data/ndis/NDIS Pricing Arrangements for Specialist Disability Accommodation 2023-24  - effective 1 July 2023.pdf', 'page': 1, 'total_pages': 49, 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Enabled': 'true', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_SetDate': '2022-03-28T22:42:56Z', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Method': 'Privileged', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Name': 'OFFICIAL', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_SiteId': 'cd778b65-752d-454a-87cf-b9990fe58993', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_ActionId': '1901f7ab-4ad4-424f-bc5a-c40559b80a1f', 'MSIP_

In [6]:
import numpy as np

print('Number of documents:', len(docs))

print(f'Average document length in characters:{np.average([len(t.page_content) for t in docs]):.1f}')
print(f'Average document length in tokens:{np.average([num_tokens_from_string(t.page_content) for t in docs]):.1f}')

Number of documents: 248
Average document length in characters:2564.4


Average document length in tokens:649.6


## text splitter

In [7]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_docs = splitter.split_documents(docs)


In [8]:

print('Number of chunks:', len(chunked_docs))

print(f'Average chunk length in characters:{np.average([len(t.page_content) for t in chunked_docs]):.1f}')
print(f'Average chunk length in tokens:{np.average([num_tokens_from_string(t.page_content) for t in chunked_docs]):.1f}')

Number of chunks: 812
Average chunk length in characters:838.8
Average chunk length in tokens:212.3


## Store in ActiveLoop deeplake 

Yet another vector store ;)  

In [9]:
# store in activeloop

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

embeddings = OpenAIEmbeddings()

# create deeplake dataset
my_activeloop_org_id = os.getenv('ACTIVE_LOOP_ORG_ID')
my_activeloop_dataset_name = ACTIVELOOP_DATASET_NAME
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(chunked_docs)


TokenPermissionError: A dataset does not exist at the specified path, or you do not have sufficient permissions to load or create one. Please check the dataset path and make sure that you have sufficient permissions to the path.

## store in chroma db

In [11]:
COLLECTION_NAME = 'NDIS_ALL_PDFPLUMBER_TEXTS_1024_128'

In [15]:
from chromadb.config import Settings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

db = Chroma.from_documents(chunked_docs, client_settings=client_settings, embedding = OpenAIEmbeddings(), collection_name=COLLECTION_NAME)

# Conversation

## load vector store as tool

In [2]:
COLLECTION_NAME = 'NDIS_ALL_PDFPLUMBER_TEXTS_1024_128'

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

chromaDb = Chroma(collection_name=COLLECTION_NAME, 
                     embedding_function=OpenAIEmbeddings(),
                     client_settings=client_settings)

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model=MODEL_NAME)

retriever = chromaDb.as_retriever()

retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

# retrieval_qa = RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type='stuff',
#     retriever=retriever,
#     # chain_type_kwargs={"prompt": PROMPT}, 
#     # return_source_documents=True,
#     verbose=True
# )

https://betterprogramming.pub/build-a-chatbot-on-your-csv-data-with-langchain-and-openai-ed121f85f0cd

https://api.python.langchain.com/en/latest/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html

In [8]:
from langchain.prompts.prompt import PromptTemplate

# https://github.com/yvann-hub/Robby-chatbot/blob/main/src/modules/chatbot.py

combine_chain_prompt_template = """
You are a helpful, polite and well-mannered bot, that answer questions from providers on how to invoice their services to the NDIS. 
You are provided with a context that contains relevant extract from the NDIS documentation.
Upon receiving the user question and context, your aim is to:
    - select for them the approriate item code from the price guide
    - determine the maximum price they can charge for the good or service according to their location
    - more generally, by advising them following recommendations set up in the price guide for that particular service if any
    
When replying, you will follow ALL of the rules below:

1/ If some information is missing to determine what item code to use, please ask that information to the user
2/ If there is more than one item code matching the given criteria, determine what makes the difference between one item code and another and ask that question to the user
3/ If you otherwise don't have enough information to answer the user query, don't invent anything and say you don't know. Do NOT try to make up an answer.
Use as much detail as possible when responding.

context: {context}
=========
question: {question}
======
        """

COMBINE_CHAIN_PROMPT = PromptTemplate(template=combine_chain_prompt_template, input_variables=["context","question" ])

### memory

In [9]:
# https://github.com/sebaxzero/Juridia/blob/main/PDF%20langchain%20example.ipynb
from langchain.memory import ConversationBufferMemory
from typing import Any, Dict
class AnswerConversationBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['answer']})
    
memory = AnswerConversationBufferMemory(memory_key="chat_history", return_messages=True)

### prompts

In [ ]:
# Prompt template: Orca-Hashes

sys_name = '### System:'
user_name = '### User:'
input_name = '### Input:'
res_name = '### Response:'


system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'

# condense question prompt

user = 'Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.'
input_text = 'Chat History:\n{chat_history}\nFollow up question: {question}'
response = 'Standalone question:'

condense_question_prompt = PromptTemplate.from_template(template=f'{sys_name}\n{system}\n\n{user_name}\n{user}\n\n{input_name}\n{input_text}\n\n{res_name}\n{response}')

# qa prompt
user = '''Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.'''
input_text = 'Question: {question}\nPieces of context:\n{context}'
response = 'Helpful Answer:'

qa_prompt = PromptTemplate.from_template(template=f'{sys_name}\n{system}\n\n{user_name}\n{user}\n\n{input_name}\n{input_text}\n\n{res_n

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

conversational_qa_chain = ConversationalRetrievalChain.from_llm(
                                                        llm=llm, 
                                                        retriever=retriever, 
                                                        verbose=True, 
                                                        return_source_documents=True, 
                                                        max_tokens_limit=4097, 
                                                        rephrase_question = True, #
                                                        memory=memory, #
                                                        combine_docs_chain_kwargs={'prompt': COMBINE_CHAIN_PROMPT})



In [11]:
def process_response(res):
    answer = res["answer"]
    source_documents = {}

    for document in res['source_documents']:
        page_content = document.page_content
        source = document.metadata['source']
        page = document.metadata['page']
        document_string = f'content: "{page_content}"'
        if source not in source_documents:
            source_documents[source] = {}
        source_documents[source][page] = document_string

    return answer, source_documents

def query(prompt: str, chain):
    res = chain({"question" : prompt})
    answer, source_documents = process_response(res=res)
    return answer, source_documents

In [12]:
question = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
answer, source_documents = query(prompt=question, chain=conversational_qa_chain)
print("Response:","\n",answer,"\n\n")

Inputs: {'question': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', 'chat_history': []}
Count Inputs: 2
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\n• Provider Travel\n• Short Notice Cancellations.\nProviders of these supports can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0120_1_1.\nThese support items are subject to price limits as set out in the following Table.\nVery\nItem Number Item Name and Notes Unit National Remote\nRemote\nHouse or Yard Maintenance $53.09 $74.33 $79.64\n• Performing essential house and/or yard\n01_019_0120_1_1 Hour\nactivities that the participant is not able to\nundertake.\nHouse Cleaning And Other Household Activities $54.07 $75.70 $81.11\n01_020_0120_1_1 • Performing essential house cleaning activities Hour\nthat the participant is not able to undertake

In [13]:
question = "Can I charge travel time with this service?"
answer, source_documents = query(prompt=question, chain=conversational_qa_chain)

Inputs: {'question': 'Can I charge travel time with this service?', 'chat_history': [HumanMessage(content='What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', additional_kwargs={}, example=False), AIMessage(content='Based on the information provided, the service you provided falls under the category of "House or Yard Maintenance". The item code for this service, according to the NDIS documentation, is 01_019_0120_1_1. \n\nAs for the price, the location of the service is in a non-remote area (Pennant Hills, NSW). The National price limit for this service is $53.09 per hour. Since you provided 2 hours of service, the maximum you can charge is $106.18. \n\nPlease ensure that this service is recorded and invoiced accurately, in compliance with NDIS Pricing Arrangements and Price Limits.', additional_kwargs={}, example=False)]}
Count Inputs: 2
Inputs: {'question': 'Can I charge trav

In [14]:
print(answer)

Based on the provided context, I can't find a specific reference to the item code 01_019_0120_1_1. Therefore, I cannot provide a definitive answer regarding whether it's possible to charge for travel time in addition to the service provided under this specific code.

However, typically, providers delivering core and capacity-building supports are permitted to claim for provider travel in respect of a support item. They can also claim for the time spent travelling from the last participant to their usual place of work, provided the provider must pay their worker for the return travel time. The maximum amount of travel time they can claim depends on the Modified Monash Model (MMM) classification of the area where the participant is when the support is delivered.

I would recommend checking the specific guidelines and rules for the item code 01_019_0120_1_1 in the NDIS Price Guide to get the most accurate information.


In [15]:
question = "Can you check the specific guidelines and rules for the item code 01_019_0120_1_1 in the NDIS Price Guide and provide the most accurate information?"
answer, source_documents = query(prompt=question, chain=conversational_qa_chain)

Inputs: {'question': 'Can you check the specific guidelines and rules for the item code 01_019_0120_1_1 in the NDIS Price Guide and provide the most accurate information?', 'chat_history': [HumanMessage(content='What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', additional_kwargs={}, example=False), AIMessage(content='Based on the information provided, the service you provided falls under the category of "House or Yard Maintenance". The item code for this service, according to the NDIS documentation, is 01_019_0120_1_1. \n\nAs for the price, the location of the service is in a non-remote area (Pennant Hills, NSW). The National price limit for this service is $53.09 per hour. Since you provided 2 hours of service, the maximum you can charge is $106.18. \n\nPlease ensure that this service is recorded and invoiced accurately, in compliance with NDIS Pricing Arrangements and Price

In [16]:
print(answer)

I'm sorry, but I don't have specific information on the item code 01_019_0120_1_1 in the NDIS Price Guide based on the provided context. For the most accurate and up-to-date information, I would recommend checking the latest version of the NDIS Price Guide and NDIS Support Catalogue on the NDIS website or the myplace provider portal. If the item code relates to a specific type of support, such as Assistive Technology or Home Modifications, you might also need to refer to the specific pricing arrangements for that type of support.


In [47]:
conversational_qa_chain_input = {"question": question, "chat_history": history}
result = conversational_qa_chain(conversational_qa_chain_input)

Inputs: {'question': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', 'chat_history': ''}
Count Inputs: 2
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\n• Provider Travel\n• Short Notice Cancellations.\nProviders of these supports can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0120_1_1.\nThese support items are subject to price limits as set out in the following Table.\nVery\nItem Number Item Name and Notes Unit National Remote\nRemote\nHouse or Yard Maintenance $53.09 $74.33 $79.64\n• Performing essential house and/or yard\n01_019_0120_1_1 Hour\nactivities that the participant is not able to\nundertake.\nHouse Cleaning And Other Household Activities $54.07 $75.70 $81.11\n01_020_0120_1_1 • Performing essential house cleaning activities Hour\nthat the participant is not able to undertake

In [48]:

from pprint import pprint
pprint(result["answer"])

('Based on the context provided, for invoicing the service of mowing the lawn, '
 'you should use the item code 01_019_0120_1_1, as this falls under the '
 'category of performing "essential house and/or yard activities that the '
 'participant is not able to undertake." \n'
 '\n'
 'The maximum price you can charge based on the location (National) is $53.09 '
 "per hour. As you've provided this service for 2 hours, you can charge a "
 'maximum of $106.18. \n'
 '\n'
 'Please make sure you follow NDIS Pricing Arrangements and Price Limits, and '
 'remember to record the service you provided using the appropriate unit of '
 'measure for the support item, in this case, hours.')


In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is Paris.")
]

### 'NDIS_COLL_PAGES_5_TEXTS_512_128' 5 pages per documents, text splitter size 512 overlap 128 with k=1

In [96]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [97]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

### 'NDIS_COLL_PAGES_5_TEXTS_512_128' 5 pages per documents, text splitter size 512 overlap 128 with k=3

In [82]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [83]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 17867 tokens. Please reduce the length of the messages.

### 'NDIS_COLL_PAGES_5_TEXTS_1024_256' with k = 2

In [75]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [76]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 17867 tokens. Please reduce the length of the messages.

### NDIS_COLLECTION with k = 2

In [58]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

# Doing my own similarity search and chain

In [135]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-16k')

template = """
You are a helpful, polite and well-mannered bot, a specialist in the NDIS Price Guide shared below.
You are striving to help providers to invoice for their services or the items they have sold to the participant.
I will share a provider's query with you - to select the right item code and determine the max price, you most generally need to know:
- the nature of the service or item sold
- the location the service was provided (postcode)
- the duration of service or the quantity of items sold
- the time of day if service (start/end)
Upon receiving the user query and the price guide context, your aim is to:
- select for them the approriate item code from the price guide
- determine the maximum price they can charge for the good or service
- more generally, by advising them following recommendations set up in the price guide for that particular service if any
When replying, you will follow ALL of the rules below:

1/ If some information is missing to determine what item code to use (location, quantity, remoteness, time of day etc), please ask that information to the user
2/ If you don't have enough information to answer the user query, don't invent anything and say you don't know

Provider query:
{query}

Here are the relevant extracts from the price guide:
{price_guide_context}

Please write the most informative answer to the provider query:
"""

prompt = PromptTemplate(
    input_variables=['query', 'price_guide_context'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [136]:

def similarity_search2(query, db):
    similar_vectors = db.similarity_search(query, k=3)

    page_contents = [doc.page_content for doc in similar_response]

    # print(page_contents_array)

    return page_contents_array

In [137]:
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings
from chromadb import Client

def get_chroma_collection(collection_name):

    client_settings = Settings(
            chroma_api_impl="rest",
            chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
            chroma_server_http_port="8000"
        )
    chromaClient = Client(client_settings)
    coll = chromaClient.get_collection(name=collection_name, embedding_function=OpenAIEmbeddings().embed_documents)
    # coll = chromaClient.get_collection(name='LANTERNPAY_COLLECTION', embedding_function=OpenAIEmbeddings().embed_documents)
    return coll

def similarity_search(query, coll, n_results=2):
    results = coll.query(query_texts=[query], n_results=2)
    metadatas = [ met for met in results['metadatas'][0]]
    docs = [ doc for doc in results['documents'][0]]
    return { 'documents': docs, 'metadatas': metadatas}

def get_query_response(query, n_results=2):
    similar_docs = similarity_search(
        query, 
        get_chroma_collection(collection_name), 
        n_results=2)
    response = chain.run(query=query, price_guide_context=similar_docs['documents'])
    return response
    

In [99]:
collection_name

'NDIS_COLL_PAGES_5_TEXTS_512_128'

In [125]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"

In [138]:

response = get_query_response(query, n_results=2)


Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', 'price_guide_context': ['Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-call assistance (either onsite or off-site) with, or\nsupervision of, personal t

'To invoice for mowing the lawn of the participant, you can use the item code 01_019_0120_1_1 - House or Yard Maintenance. This item code covers performing essential house and/or yard activities that the participant is not able to undertake. The unit for this item code is an hour. The maximum price you can charge for this service depends on the location. In Pennant Hills, NSW, the maximum price for this service is $79.64. \n\nPlease note that the price guide also provides information on claiming for provider travel. If you incurred any travel costs while providing the service, you may be able to claim for those as well. The specific item code for provider travel - non-labour costs is 01_799_0120_1_1. However, please ensure that the conditions for claiming provider travel are met as outlined in the price guide.\n\nIf you have any further questions or need additional information, please let me know.'

In [139]:
print(response)

To invoice for mowing the lawn of the participant, you can use the item code 01_019_0120_1_1 - House or Yard Maintenance. This item code covers performing essential house and/or yard activities that the participant is not able to undertake. The unit for this item code is an hour. The maximum price you can charge for this service depends on the location. In Pennant Hills, NSW, the maximum price for this service is $79.64. 

Please note that the price guide also provides information on claiming for provider travel. If you incurred any travel costs while providing the service, you may be able to claim for those as well. The specific item code for provider travel - non-labour costs is 01_799_0120_1_1. However, please ensure that the conditions for claiming provider travel are met as outlined in the price guide.

If you have any further questions or need additional information, please let me know.


In [127]:
similar_docs.keys()

dict_keys(['documents', 'metadatas'])

In [122]:
# print(similar_docs['documents'][0][1])
print(similar_docs['metadatas'][0][0])

{'title': 'NDIS Pricing Arrangements and Price Limits 2023-24', 'source': '/workspaces/IsabelleLangchain/src/ndis/../../data/ndis2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'timestamp': '2023-08-06T09:01:46.122650', 'start_page': 41, 'end_page': 45}
